# FUNCTIONS AND CONSTANTS
in this code block, all imports, functions and constants are defined

In [1]:
import serial       # communication with the board
import numpy as np  # science
import pandas as pd # more science
import struct       # needed for CRC Check (unpacking bytes to array)
import time         # to sleep
import matplotlib.pyplot as plt


SYSTEM_VERBOSE = 0
SERIAL_PSU  = 'COM12'
SERIAL_PG   = 'COM11'

FILE_PATH       =   "/data_csv/"
FILE_NAME_PSU   =   "psu_data.csv"
FILE_NAME_PG    =   "pg_data.csv"

df_save_frequency = 2
send_attempts = 5

# PSU_STATE: 0 --> AFTER STARTUP, 1 --> AFTER SETUP, 2 --> AFTER RUN, 3 --> AFTER STOP
csv_header_psu = ['crc', 'type', 'status', 'p_setpoint', 'p_value', 'n_setpoint', 'n_value', 'p_current', 'n_current','psu_state', 'p_sent', 'n_sent', 'time']  # PSU STATE IS JUST A FLAG FOR WHAT I DID
#psu_datatypes = {'crc':'int32', 'type':'int32', 'status':'int32', 'p_setpoint':'int32', 'p_value':'int32', 'n_setpoint':'int32', 'n_value':'int32', 'p_current':'int32', 'n_current':'int32', 'psu_state':'int32', 'p_sent':'int32', 'n_sent':'int32', 'time':'float32'}

csv_header_pg = ['voltage_1_A', 'voltage_1_B', 'current_1_A', 'current_1_B', 'type', 'length', 'zero_volt', 'zero_curr', 'data_start', 'data_end']

# ===============================================================
# ==================== FUNCTIONS: PRINT STUFF ===================
# ===============================================================

def printSerialData_asBinary(serialData):
    binary_repr = ' '.join(format(byte, '08b') for byte in serialData)
    print("DATA AS BINARY:\t{}".format(binary_repr))

def printSerialData_asHexadecimal(serialData):
    print("DATA AS HEX:\t{}".format(serialData))

def printSerialData_asIntegerArray(serialData):
    arr = np.array(struct.unpack('<' + 'H' * (len(serialData) // 2), serialData))
    print("DATA AS ARRAY:\t{}".format(arr))

def printArray_asBinary(arrayData, bitSize=8):
    print("BINARY REPRESENTATION:")
    for elem in arrayData:
        print(np.binary_repr(elem, width=bitSize), end=" ")
    print("")


# ===============================================================
# ======================== FUNCTIONS: CRC =======================
# ===============================================================

# CRC TABLE FOR CRC CALCULATION
crc16_table = [
    0x0000, 0xC0C1, 0xC181, 0x0140, 0xC301, 0x03C0, 0x0280, 0xC241,
	0xC601, 0x06C0, 0x0780, 0xC741, 0x0500, 0xC5C1, 0xC481, 0x0440,
	0xCC01, 0x0CC0, 0x0D80, 0xCD41, 0x0F00, 0xCFC1, 0xCE81, 0x0E40,
	0x0A00, 0xCAC1, 0xCB81, 0x0B40, 0xC901, 0x09C0, 0x0880, 0xC841,
	0xD801, 0x18C0, 0x1980, 0xD941, 0x1B00, 0xDBC1, 0xDA81, 0x1A40,
	0x1E00, 0xDEC1, 0xDF81, 0x1F40, 0xDD01, 0x1DC0, 0x1C80, 0xDC41,
	0x1400, 0xD4C1, 0xD581, 0x1540, 0xD701, 0x17C0, 0x1680, 0xD641,
	0xD201, 0x12C0, 0x1380, 0xD341, 0x1100, 0xD1C1, 0xD081, 0x1040,
	0xF001, 0x30C0, 0x3180, 0xF141, 0x3300, 0xF3C1, 0xF281, 0x3240,
	0x3600, 0xF6C1, 0xF781, 0x3740, 0xF501, 0x35C0, 0x3480, 0xF441,
	0x3C00, 0xFCC1, 0xFD81, 0x3D40, 0xFF01, 0x3FC0, 0x3E80, 0xFE41,
	0xFA01, 0x3AC0, 0x3B80, 0xFB41, 0x3900, 0xF9C1, 0xF881, 0x3840,
	0x2800, 0xE8C1, 0xE981, 0x2940, 0xEB01, 0x2BC0, 0x2A80, 0xEA41,
	0xEE01, 0x2EC0, 0x2F80, 0xEF41, 0x2D00, 0xEDC1, 0xEC81, 0x2C40,
	0xE401, 0x24C0, 0x2580, 0xE541, 0x2700, 0xE7C1, 0xE681, 0x2640,
	0x2200, 0xE2C1, 0xE381, 0x2340, 0xE101, 0x21C0, 0x2080, 0xE041,
	0xA001, 0x60C0, 0x6180, 0xA141, 0x6300, 0xA3C1, 0xA281, 0x6240,
	0x6600, 0xA6C1, 0xA781, 0x6740, 0xA501, 0x65C0, 0x6480, 0xA441,
	0x6C00, 0xACC1, 0xAD81, 0x6D40, 0xAF01, 0x6FC0, 0x6E80, 0xAE41,
	0xAA01, 0x6AC0, 0x6B80, 0xAB41, 0x6900, 0xA9C1, 0xA881, 0x6840,
	0x7800, 0xB8C1, 0xB981, 0x7940, 0xBB01, 0x7BC0, 0x7A80, 0xBA41,
	0xBE01, 0x7EC0, 0x7F80, 0xBF41, 0x7D00, 0xBDC1, 0xBC81, 0x7C40,
	0xB401, 0x74C0, 0x7580, 0xB541, 0x7700, 0xB7C1, 0xB681, 0x7640,
	0x7200, 0xB2C1, 0xB381, 0x7340, 0xB101, 0x71C0, 0x7080, 0xB041,
	0x5000, 0x90C1, 0x9181, 0x5140, 0x9301, 0x53C0, 0x5280, 0x9241,
	0x9601, 0x56C0, 0x5780, 0x9741, 0x5500, 0x95C1, 0x9481, 0x5440,
	0x9C01, 0x5CC0, 0x5D80, 0x9D41, 0x5F00, 0x9FC1, 0x9E81, 0x5E40,
	0x5A00, 0x9AC1, 0x9B81, 0x5B40, 0x9901, 0x59C0, 0x5880, 0x9841,
	0x8801, 0x48C0, 0x4980, 0x8941, 0x4B00, 0x8BC1, 0x8A81, 0x4A40,
	0x4E00, 0x8EC1, 0x8F81, 0x4F40, 0x8D01, 0x4DC0, 0x4C80, 0x8C41,
	0x4400, 0x84C1, 0x8581, 0x4540, 0x8701, 0x47C0, 0x4680, 0x8641,
	0x8201, 0x42C0, 0x4380, 0x8341, 0x4100, 0x81C1, 0x8081, 0x4040
]

# FUNCTION FOR CRC CALCULATION (Python implementation of Hans' Code)
# BUFFER SHOULD BE A UINT8-ARRAY
def crc16(crc, buffer_uint8):
    for byte in buffer_uint8:
        crc = (crc >> 8) ^ crc16_table[(crc ^ byte) & 0xff]
    return crc

# FUNCTION TO TEST THE CRC OF A COMPLETE DATA PACKAGE
def testCRC(binaryData, verbose=0):
    if verbose: print("----------------- START: TEST CRC -----------------")        # PRINT FORMATTING

    crc_check = False
    # GET AND CALCULATE THE CRC VALUES
    arr = np.array(struct.unpack('<' + 'H' * (len(binaryData) // 2), binaryData))   # CREATE DATA ARRAY (uint16) 
    dataCRC = arr[0]                                                                # GET CRC VALUE FROM THE ARRAY
    calcCRC = crc16(0,  np.frombuffer(binaryData[2:], dtype=np.uint8))              # CALCULATE CRC VALUE ACCORDING TO DATA

    if verbose >1: print("DATA AS ARRAY:\n{}".format(arr))                          # PRINT FORMATTING
    if verbose: print("CRC IN DATA:\t\t{}".format(dataCRC))                         # PRINT FORMATTING
    if verbose: print("CRC CALCULATED:\t{}".format(calcCRC))                        # PRINT FORMATTING
    
    # CRC CHECK
    if dataCRC == calcCRC:
        crc_check = True
    
    if verbose: print("CRC CHECK:\t\t{}".format(crc_check))                         # PRINT FORMATTING
    if verbose: print("----------------- END: TEST CRC -----------------")          # PRINT FORMATTING
    
    # RETURN DATA
    return crc_check, calcCRC


# ===============================================================
# ==================== FUNCTIONS: READ/WRITE SERIAL ===================
# ===============================================================

# FUNCTION TO READ AND PRINT SERIAL DATA FROM CONNECTED DEVICE
def readSerialData(ser, verbose=0):
    if verbose: print("==================== START: READING ====================")      # PRINT FORMATTING

    serialData = ser.read(64)                                   # READ 64 BYTES OF THE DATA STREAM (= 1 PACKET)
    crc_status, crc_value = testCRC(serialData, verbose)        # CHECK THE CRC AND RETURN THE CRC-VALUE

    if verbose >1: printSerialData_asBinary(serialData)         # PRINT FORMATTING
    if verbose >1: printSerialData_asHexadecimal(serialData)    # PRINT FORMATTING
    if verbose >1: printSerialData_asIntegerArray(serialData)   # PRINT FORMATTING

    # SHOW ERROR MESSAGE, IF CRC IS WRONG
    if not crc_status:
        print("CRC Error with received data!")

    if verbose: print("==================== END: READING ====================")      # PRINT FORMATTING
    return serialData, crc_status, crc_value

# FUNCTION TO TRY WRITING TO SERIAL X TIMES
def writeSerialData(ser, data_to_send, num_attempts, verbose=0):
    for i in range(num_attempts):
        try:
            ser.write(data_to_send)
            if verbose: print('Data sent successfully')
            break  # if data was sent successfully, exit the loop
        except serial.SerialTimeoutException:
            if verbose: print(f'Timeout occurred while sending data. Attempt {i+1}/{num_attempts}')
            time.sleep(0.5)  # wait for a short time before attempting again


# ===============================================================
# =============== FUNCTIONS: PREPARE DATA PACKETS ===============
# ===============================================================

# CRAFT SETPOINT PACKAGE FOR PSU
def PSU_CraftPackage_setSetpoints(posVoltVal, negVoltVal, verbose=0):
    payload = np.array([
        0x0001,             # SetSetpoints Command 
        posVoltVal,         # posValue = 10V
        abs(negVoltVal),    # negValue = -10V (abs value)
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 28), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata

# CRAFT RUN PACKAGE FOR PSU
def PSU_CraftPackage_run(verbose=0):
    payload = np.array([
        0x0003              # Start Command 
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 30), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata

# CRAFT STOP PACKAGE FOR PSU
def PSU_CraftPackage_stop(verbose=0):
    payload = np.array([
        0x0004              # Stop Command 
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 30), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata

# CRAFT PULSE PACKAGE FOR PG
def PG_CraftPackage_setTimes(repRate=5000, frequency=0 ,pulseLength=75, onTime=248, verbose=0):
    '''
    Returns the crafted byte-packet including crc. READY TO SEND.

        Parameters:
                repRate (int):      Time between pulses (us)                [750 .. inf]
                                    DEFAULT: 5000 (every 5000us --> 200Hz)

                frequency (int):    Frequency for pulses (HZ)               [1 .. 1300]
                                    DEFAULT: 0 
                                    NOTE: (if it is set, this value has more importance than repRate!)

                pulseLength (int):  Length of the positive pulse (us)       [10 .. 250]
                                    DEFAULT: 75

                onTime (int):       Time of the Transistor being ON (us)    [5 .. pulseLength-2]
                                    DEFAULT: 248 (is the maximum possible value. Gets capped anyways due to the pulseLength)

                verbose (int):      0.. printing OFF, 1.. basic printing, 2.. print ALL
                                    DEFAULT: 

        Returns:
                sendable_bytedata (str): final byte packet
    '''
    # CALCULATE THE repRate IF THE FREQUENCY IS SET (from HZ to us)
    if frequency:
        repRate = 1e6 / frequency


    # PACKET AS ARRAY
    payload = np.array([
        0x0001,             # SetSetpoints Command 
        repRate,            # Frequency --> How many pulses per second
        pulseLength,        # Length of the ON-Pulse (also scales the whole pulse)
        onTime,             # Something for clipping. Set it to the repRate to be save
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 27), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata

# RUN AND STOP ARE THE SAME, HOWEVER I USED TWO SEPARATE CREATE FUNCTIONS, JUST TO BE SURE
def PG_CraftPackage_run(verbose=0):
    payload = np.array([
        0x0003              # Start Command 
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 30), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata

def PG_CraftPackage_stop(verbose=0):
    payload = np.array([
        0x0004              # Stop Command 
        ], dtype=np.uint16) # data type as uint16 (16 bit)

    # FILL IT WITH 0s THAT IT HAS A SIZE OF 31 UINT16 (1 left for CRC)
    datapacket = np.pad(payload, (0, 30), mode='constant')

    # CREATE A BYTE REPRESENTATION FOR CRC CALCULATION
    byte_datapacket = datapacket.tobytes()

    # CALCULATE CRC
    crc = crc16(0,  np.frombuffer(byte_datapacket, dtype=np.uint8))

    # CREATE COMPLETE ARRAY FOR USB DATA
    sendable_arr = np.insert(datapacket, 0, crc)

    # CREATE A BYTE REPRESENTATION FOR USB DATA
    sendable_bytedata = sendable_arr.tobytes()
    if verbose: print(sendable_bytedata)

    # TEST THE CRC
    testCRC(sendable_bytedata, verbose)

    return sendable_bytedata



# PROGRAM TO TEST THE PSU RESPONSE

This chapter tests the response to the setpoints, enable and disable.

**Setpoints:** (30V/-20V) --> Enable --> (15V/-10V) --> 0V --> Disable

In [ ]:
# ##############################################################################################################
# ================================================ MAIN PROGRAM ================================================
# ##############################################################################################################

# VARIABLES
posVolt = 30
negVolt = -20

# DATAFRAME
df_psu = pd.DataFrame(columns=csv_header_psu)

# SETUP SERIAL CONNECTION
serPSU = serial.Serial(SERIAL_PSU, 9600)
serPG = serial.Serial(SERIAL_PG, 9600)

# PSU OFF
PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)

time.sleep(2)


# ================================================ DO THE THINGS ================================================

# READ STARTUP
for i in range(100):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 0
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



# PSU SET SETPOINTS to +-12V
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt, negVolt, SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)
time.sleep(2)

# READ SETDATA
for i in range(100):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 1
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



# PSU SET ENABLE
PSU_sendablePacket = PSU_CraftPackage_run(SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)
time.sleep(2)

# READ ON
for i in range(2000):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 2
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



# PSU SET SETPOINTS
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt/2, negVolt/2, SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)
time.sleep(2)

# READ CHANGED
for i in range(2000):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 2
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



# PSU SET SETPOINTS
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(1, 1, SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)
time.sleep(2)

# READ CHANGED 2
for i in range(2000):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 2
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



# PSU OFF
PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
serPSU.write(PSU_sendablePacket)

# READ OFF
for i in range(5000):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    #print(PSU_serData)
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 2
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME





print("PSU DATAFRAME")
print(df_psu)

df_psu.plot(y=['p_setpoint', 'p_value'], figsize=(10,5))
df_psu.plot(y=['n_setpoint', 'n_value'], figsize=(10,5))
df_psu.plot(y=['p_value', 'n_value'], figsize=(10,5))
df_psu.plot(y=['p_setpoint', 'p_value', 'n_setpoint', 'n_value'], figsize=(10,5))


serPSU.close()
serPG.close()


# PROGRAM TO TEST THE SETPOINT & VOLTAGE CORRELATION (SLOPE & OFFSET)

This chapter tests the response to the setpoints and the correlation between setpoints and voltage

**Setpoints:** linear slope from +-0 to +-70 (Stepsize 1)

In [ ]:
# ##############################################################################################################
# ================================================ MAIN PROGRAM ================================================
# ##############################################################################################################

# DATAFRAME
df_psu = pd.DataFrame(columns=csv_header_psu)

# SETUP SERIAL CONNECTION
serPSU = serial.Serial(SERIAL_PSU, 9600, write_timeout=1)
serPG = serial.Serial(SERIAL_PG, 9600, write_timeout=1)

# PSU OFF
PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(10)


# ================================================ DO THE THINGS ================================================

# READ STARTUP
print("READ STUFF")
for i in range(100):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:10]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 0
    #if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME


# SET TO ZERO
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(0, 0, SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

# PSU SET ENABLE
PSU_sendablePacket = PSU_CraftPackage_run(SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)
time.sleep(10)

start_time = time.time()

# 0 - 90V TEST
for value in range(0, 91, 5):
    posVolt = value
    negVolt = -value
    PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt, negVolt, SYSTEM_VERBOSE)
    writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)
    #time.sleep(2)

    for i in range(500):
        PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
        arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:13]                # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
        arr[9] = 0
        arr[10] = posVolt
        arr[11] = negVolt
        arr[12] = (time.time() - start_time)*1000
        if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME


# 25 - 0V TEST
#for value in range(25, -1, -1):
#    posVolt = value
#    negVolt = -value
#    PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt, negVolt, SYSTEM_VERBOSE)
#    writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)
#    time.sleep(1)

#    for i in range(200):
#        PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
#        arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:13]                # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
#        arr[9] = 0
#        arr[10] = posVolt
#        arr[11] = negVolt
#        arr[12] = (time.time() - start_time)*1000
#        if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME





# PSU OFF
PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
#serPSU.write(PSU_sendablePacket)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

# READ OFF
for i in range(4000):
    PSU_serData, PSU_serData_crcStatus, PSU_serData_crcValue = readSerialData(serPSU, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    arr = np.array(struct.unpack('<' + 'H' * (len(PSU_serData) // 2), PSU_serData))[:13]              # CONSTRUCT UINT16 ARRAY AND JUST KEEP THE FIRST 9 ENTRIES (EVERYTHING ELSE IS 0 ANYWAYS)
    arr[9] = 2
    arr[10] = 0
    arr[11] = 0
    arr[12] = (time.time() - start_time)*1000
    if (not (i % df_save_frequency)): df_psu = pd.concat([df_psu, pd.DataFrame([arr], columns=csv_header_psu)], ignore_index=True)    # ADD IT TO THE DATAFRAME



serPSU.close()
serPG.close()


In [ ]:
# PLOT ALL THE STUFF
df_psu.plot(
    title="PSU received data: positive setpoint and value",
    x='time',
    y=['p_setpoint', 'p_value'],
    figsize=(10,5),
    color=['tomato', 'red'],
    style=[':', '-'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU received data: negative setpoint and value",
    x='time',
    y=['n_setpoint', 'n_value'],
    figsize=(10,5),
    color=['cornflowerblue', 'blue'],
    style=[':', '-'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU: positive setpoint and previously sent data",
    x='time',
    y=['p_setpoint', 'p_sent'],
    figsize=(10,5),
    color=['tomato', 'maroon'],
    style=[':', '--'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU: negative setpoint and previously sent data",
    x='time',
    y=['n_setpoint', 'n_sent'],
    figsize=(10,5),
    color=['cornflowerblue', 'midnightblue'],
    style=[':', '--'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU received data: positive & negative value",
    x='time',
    y=['p_value', 'n_value'],
    figsize=(10,5),
    color=['red', 'blue'],
    style=['-', '-'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU received data: positive & negative values and setpoints",
    x='time',
    y=['p_setpoint', 'p_value', 'n_setpoint', 'n_value'],
    figsize=(10,5),
    color=['tomato', 'red', 'cornflowerblue', 'blue'],
    style=[':', '-', ':', '-'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')

df_psu.plot(
    title="PSU received data: positive & negative values and setpoints",
    x='time',
    y=['p_setpoint', 'p_value', 'n_setpoint', 'n_value'],
    figsize=(10,5),
    color=['tomato', 'red', 'cornflowerblue', 'blue'],
    style=[':', '-', ':', '-'],
    grid=True,
    )
#plt.xlabel('samples')
plt.xlabel('time [ms]')
plt.ylabel('voltage [V]')


df_setpoints = df_psu[(df_psu[['p_setpoint', 'n_setpoint']] != 0).all(axis=1)]


df_setpoints.iloc[50:140000].plot.scatter(
    title="PSU received data: positive setpoint data over sent data",
    x='p_sent',
    y='p_setpoint',
    figsize=(10,5),
    color='red',
    grid=True,
    )
plt.xlabel('setpoint [V]')
plt.ylabel('received value')

df_setpoints.iloc[50:140000].plot.scatter(
    title="PSU received data: negative setpoint data over sent data",
    x='n_sent',
    y='n_setpoint',
    figsize=(10,5),
    color='blue',
    grid=True,
    )
plt.xlabel('setpoint [V]')
plt.ylabel('received value')

plt.show()

df_setpoints

In [ ]:
df_psu.to_csv('data_csv/psu_linear_data_270423_1241.csv', index=False)

In [ ]:
df_psu.head()

In [ ]:
df_psu.tail(20)

# PROGRAM TO READ THE PULSES

This chapter tests the response of the Pulse Generator

**Setpoints:** +-45V

In [2]:
# ##############################################################################################################
# ================================================ MAIN PROGRAM ================================================
# ##############################################################################################################

posVolt = 40
negVolt = 42

repRate = 5000              # PULSE EVERY ... MICROSECONDS
pulseLength = 75            # TIME THAT THE PULSE IS ON (MICROSECONDS)
onTime = 75                 # WHATEVER.. GETS CAPPED ANYWAYS


zeroVoltage = -1
zeroCurrent = -1

# ##############################################################################################################
# ================================================ IN HANS' CODE ===============================================
# ##############################################################################################################

# --------------- SetOnTime --------------- (3)
#   if (OnTime > PulseLength - 2)
#       OnTime = PulseLength - 2;
#	if (OnTime < 5)
#	    OnTime = 5;

# ------------ SetPulsePeriod ------------- (1)
#   if (pulsePeriod < 750)
#       PulsePeriod = 1000;
#	if (OnTime > 10000)     WTF.. WHY? IT WILL BE SET TO BE PulseLength-2 ANYWAYS
#		OnTime = 10000;
#	if (OnTime < 2000)
#		OnTime = 2000;

# ------------ SetPulseLength ------------- (2)
#   if (PulseLength > 250)
#		PulseLength = 250;
#	if (PulseLength < 10)
#		PulseLength = 10;








# DATAFRAME
df_pg = pd.DataFrame(columns=csv_header_pg)

# SETUP SERIAL CONNECTION
serPSU = serial.Serial(SERIAL_PSU, 9600, write_timeout=1)
serPG = serial.Serial(SERIAL_PG, 9600, write_timeout=1)

# SET PSU VOLTAGE
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt, negVolt, SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)

# PSU ON
PSU_sendablePacket = PSU_CraftPackage_run(SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)


# ================================================ PULSE GENERATOR ================================================

# SET TIMES AND ENABLE
PG_sendablePacket = PG_CraftPackage_setTimes(repRate=repRate, pulseLength=pulseLength, onTime=onTime, verbose=SYSTEM_VERBOSE) #ADD STUFF
writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)

PG_sendablePacket = PG_CraftPackage_run(SYSTEM_VERBOSE)
writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

# READ SERIAL DATA, UNTIL ZERO VOLTAGE/CURRENT IS RECEIVED
while (zeroVoltage == -1):
    PG_serData, PG_serData_crcStatus, PG_serData_crcValue = readSerialData(serPG, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    crc, status = struct.unpack('<HH', PG_serData[:4]) # TAKE THE FIRST 4 BYTES AND UNPACK THEM AS 2x UINT16
    if status == 0x1003:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        arr = np.array(struct.unpack(data_fmt, PG_serData[4:]), dtype=np.uint16)
        zeroVoltage = arr[0]
        zeroCurrent = arr[1]
        df_pg = pd.concat([df_pg, pd.DataFrame([[zeroVoltage, zeroCurrent, 'zero']], columns=['zero_volt', 'zero_curr', 'type'])], ignore_index=True)
        




# MEASURE
for i in range(10000):
    PG_serData, PG_serData_crcStatus, PG_serData_crcValue = readSerialData(serPG, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    crc, status = struct.unpack('<HH', PG_serData[:4]) # TAKE THE FIRST 4 BYTES AND UNPACK THEM AS 2x UINT16

    # CHECK THE STATUS AND PACK IT ACCORDINGLY (SEE TABLE FOR DATA STRUCTURE)
    # data_type: 0.. START, 1.. END, 2.. DATA, 3.. ZERO DATA
    if status == 0x1000:
        data_fmt = '<' + 'I'*15         # 15 TIMES UINT32
        data_size = 15                  
        data_type = 0

    elif status == 0x1001:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = 1

    elif status == 0x1002:              # 60 TIMES UINT8
        data_fmt = '<' + 'B'*60
        data_size = 60
        data_type = 2

    elif status == 0x1003:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = 3

    else:
        #raise ValueError(f'Invalid status value: {status}')
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = -1
        print('Invalid status value: {}'.format(status))

    # CREATE THE ARRAY (UINT8/16/32 DEPENDING ON THE SIZE)
    arr = np.array(struct.unpack(data_fmt, PG_serData[4:]), dtype=np.uint32 if data_size == 15 else np.uint8 if data_size == 60 else np.uint16)
    
    # ['voltage_1_A', 'voltage_1_B', 'current_1_A', 'current_1_B', 'type', 'length', 'zero_volt', 'zero_curr', 'data_start', 'data_end']


    # PULSE DATA START
    if data_type == 0:
        df_pg = pd.concat([df_pg, pd.DataFrame([[arr[0], 1, 'start']], columns=['length', 'data_start', 'type'])], ignore_index=True)

    # PULSE DATA END
    elif data_type == 1:
        df_pg = pd.concat([df_pg, pd.DataFrame([[1, 'end']], columns=['data_end', 'type'])], ignore_index=True)

    # PULSE DATA
    # RESHAPE THE ARRAY IN CASE IT IS JUST DATA (0x1002) TO 4 COLUMNS (V1, V2, I1, I2)
    elif data_type == 2:
        arr_reshaped = arr.reshape(-1, 4)

        #arr[12] = (time.time() - start_time)*1000
        df_pg = pd.concat([df_pg, pd.DataFrame(arr_reshaped, columns=['voltage_1_A', 'voltage_1_B', 'current_1_A', 'current_1_B'])], ignore_index=True)    # ADD IT TO THE DATAFRAME

    # PULSE DATA ZERO DATA
    elif data_type == 3:
        df_pg = pd.concat([df_pg, pd.DataFrame([[arr[0], arr[1], 'zero']], columns=['zero_volt', 'zero_curr', 'type'])], ignore_index=True)  
   

#PG_sendablePacket = PG_CraftPackage_stop(SYSTEM_VERBOSE)
#writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

#PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
#writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

serPSU.close()
serPG.close()






# DATA PREPARATION
# FILL ALL NAN
df_pg.fillna(0, inplace=True)


Invalid status value: 4369
Invalid status value: 4369


# TEST WITH DATA = UINT16

In [ ]:
# ##############################################################################################################
# ================================================ MAIN PROGRAM ================================================
# ##############################################################################################################

posVolt = 45
negVolt = -45

repRate = 5000              # PULSE EVERY ... MICROSECONDS
pulseLength = 75            # TIME THAT THE PULSE IS ON (MICROSECONDS)
onTime = 75                 # WHATEVER.. GETS CAPPED ANYWAYS


zeroVoltage = -1
zeroCurrent = -1

# ##############################################################################################################
# ================================================ IN HANS' CODE ===============================================
# ##############################################################################################################

# --------------- SetOnTime --------------- (3)
#   if (OnTime > PulseLength - 2)
#       OnTime = PulseLength - 2;
#	if (OnTime < 5)
#	    OnTime = 5;

# ------------ SetPulsePeriod ------------- (1)
#   if (pulsePeriod < 750)
#       PulsePeriod = 1000;
#	if (OnTime > 10000)     WTF.. WHY? IT WILL BE SET TO BE PulseLength-2 ANYWAYS
#		OnTime = 10000;
#	if (OnTime < 2000)
#		OnTime = 2000;

# ------------ SetPulseLength ------------- (2)
#   if (PulseLength > 250)
#		PulseLength = 250;
#	if (PulseLength < 10)
#		PulseLength = 10;








# DATAFRAME
df_pg = pd.DataFrame(columns=csv_header_pg)

# SETUP SERIAL CONNECTION
serPSU = serial.Serial(SERIAL_PSU, 9600, write_timeout=1)
serPG = serial.Serial(SERIAL_PG, 9600, write_timeout=1)

# SET PSU VOLTAGE
PSU_sendablePacket = PSU_CraftPackage_setSetpoints(posVolt, negVolt, SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)

# PSU ON
PSU_sendablePacket = PSU_CraftPackage_run(SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)


# ================================================ PULSE GENERATOR ================================================

# SET TIMES AND ENABLE
PG_sendablePacket = PG_CraftPackage_setTimes(repRate=repRate, pulseLength=pulseLength, onTime=onTime, verbose=SYSTEM_VERBOSE) #ADD STUFF
writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

time.sleep(2)

PG_sendablePacket = PG_CraftPackage_run(SYSTEM_VERBOSE)
writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

# READ SERIAL DATA, UNTIL ZERO VOLTAGE/CURRENT IS RECEIVED
while (zeroVoltage == -1):
    PG_serData, PG_serData_crcStatus, PG_serData_crcValue = readSerialData(serPG, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    crc, status = struct.unpack('<HH', PG_serData[:4]) # TAKE THE FIRST 4 BYTES AND UNPACK THEM AS 2x UINT16
    if status == 0x1003:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        arr = np.array(struct.unpack(data_fmt, PG_serData[4:]), dtype=np.uint16)
        zeroVoltage = arr[0]
        zeroCurrent = arr[1]
        df_pg = pd.concat([df_pg, pd.DataFrame([[zeroVoltage, zeroCurrent, 'zero']], columns=['zero_volt', 'zero_curr', 'type'])], ignore_index=True)
        




# MEASURE
for i in range(1000):
    PG_serData, PG_serData_crcStatus, PG_serData_crcValue = readSerialData(serPG, SYSTEM_VERBOSE)   # READ SERIAL DATA FROM PSU
    crc, status = struct.unpack('<HH', PG_serData[:4]) # TAKE THE FIRST 4 BYTES AND UNPACK THEM AS 2x UINT16

    # CHECK THE STATUS AND PACK IT ACCORDINGLY (SEE TABLE FOR DATA STRUCTURE)
    # data_type: 0.. START, 1.. END, 2.. DATA, 3.. ZERO DATA
    if status == 0x1000:
        data_fmt = '<' + 'I'*15         # 15 TIMES UINT32
        data_size = 15                  
        data_type = 0

    elif status == 0x1001:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = 1

    elif status == 0x1002:              # 60 TIMES UINT8 (NOW ALSO 30 TIMES UINT16)
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = 2

    elif status == 0x1003:               # 30 TIMES UINT16
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = 3

    else:
        #raise ValueError(f'Invalid status value: {status}')
        data_fmt = '<' + 'H'*30
        data_size = 30
        data_type = -1
        print('Invalid status value: {}'.format(status))

    # CREATE THE ARRAY (UINT8/16/32 DEPENDING ON THE SIZE)
    arr = np.array(struct.unpack(data_fmt, PG_serData[4:]), dtype=np.uint32 if data_size == 15 else np.uint8 if data_size == 60 else np.uint16)
    
    # ['voltage_1_A', 'voltage_1_B', 'current_1_A', 'current_1_B', 'type', 'length', 'zero_volt', 'zero_curr', 'data_start', 'data_end']


    # PULSE DATA START
    if data_type == 0:
        df_pg = pd.concat([df_pg, pd.DataFrame([[arr[0], 1, 'start']], columns=['length', 'data_start', 'type'])], ignore_index=True)

    # PULSE DATA END
    elif data_type == 1:
        df_pg = pd.concat([df_pg, pd.DataFrame([[1, 'end']], columns=['data_end', 'type'])], ignore_index=True)

    # PULSE DATA
    # RESHAPE THE ARRAY IN CASE IT IS JUST DATA (0x1002) TO 4 COLUMNS (V1, V2, I1, I2)
    elif data_type == 2:
        arr_reshaped = arr.reshape(-1, 2)

        #arr[12] = (time.time() - start_time)*1000
        df_pg = pd.concat([df_pg, pd.DataFrame(arr_reshaped, columns=['voltage', 'current'])], ignore_index=True)    # ADD IT TO THE DATAFRAME

    # PULSE DATA ZERO DATA
    elif data_type == 3:
        df_pg = pd.concat([df_pg, pd.DataFrame([[arr[0], arr[1], 'zero']], columns=['zero_volt', 'zero_curr', 'type'])], ignore_index=True)  
   

PG_sendablePacket = PG_CraftPackage_stop(SYSTEM_VERBOSE)
writeSerialData(serPG, PG_sendablePacket, send_attempts, SYSTEM_VERBOSE)

PSU_sendablePacket = PSU_CraftPackage_stop(SYSTEM_VERBOSE)
writeSerialData(serPSU, PSU_sendablePacket, send_attempts, SYSTEM_VERBOSE)

serPSU.close()
serPG.close()






# DATA PREPARATION
# FILL ALL NAN
df_pg.fillna(0, inplace=True)


# PLOT ALL THE STUFF
df_pg.plot(
    title="PG received data: voltage",
    y=['voltage', 'current'],
    figsize=(15,5),
    color=['red', 'blue'],
    style=['-', '-'],
    grid=True,
    )
plt.xlabel('samples')
#plt.xlabel('time [ms]')
plt.ylabel('value')


plt.show()

In [ ]:
# PLOT ALL THE STUFF
df_pg.plot(
    title="PG received data: voltage",
    y=['voltage_1_A', 'voltage_1_B'],
    figsize=(15,5),
    color=['red', 'blue'],
    style=['-', '-'],
    grid=True,
    )
plt.xlabel('samples')
#plt.xlabel('time [ms]')
plt.ylabel('value')

# PLOT ALL THE STUFF
df_pg.plot(
    title="PG received data: current",
    y=['current_1_A', 'current_1_B'],
    figsize=(15,5),
    color=['green', 'purple'],
    style=['-', '-'],
    grid=True,
    )
plt.xlabel('samples')
#plt.xlabel('time [ms]')
plt.ylabel('value')

plt.show()

In [ ]:
df_pg.to_csv('data_csv/pg_data_020523_1201.csv', index=False)

In [ ]:
# WHICH PULSE NUMBER DO YOU WANT?
n = 1
print_single = False
nr_of_prints = 10

# SEARCH FOR INDEX FOR ALL START END END SIGNALS
df_start = df_pg[df_pg['type'] == 'start']
start_index_arr = df_start.index.tolist()

df_end = df_pg[df_pg['type'] == 'end']
end_index_arr = df_end.index.tolist()

# CREATE DATAFRAME (EMPTY)
df_times = pd.DataFrame(columns=['start', 'end'])

# ARRANGE AND COMBINE START AND END SIGNALS INSIDE THE DATASET
for start_index in start_index_arr:
    for end_index in end_index_arr:
        if end_index > start_index:
            df_times = pd.concat([df_times, pd.DataFrame([[start_index, end_index]], columns=['start', 'end'])], ignore_index=True)
            break


# GET THE Nth PULSE


if print_single:
    # GET THE Nth PULSE
    df_firstpulse = df_pg.iloc[df_times.iloc[n]['start']:df_times.iloc[n]['end']]
    
    # PLOT ALL THE STUFF
    df_firstpulse.plot(
        title="SINGLE PULSE (n={}): VOLTAGE".format(n),
        y=['voltage_1_A', 'voltage_1_B'],
        figsize=(15,5),
        color=['red', 'blue'],
        style=['-', '-'],
        grid=True,
        )
    plt.xlabel('samples')
    #plt.xlabel('time [ms]')
    plt.ylabel('value')

    # PLOT ALL THE STUFF
    df_firstpulse.plot(
        title="SINGLE PULSE (n={}): VOLTAGE_1_B".format(n),
        y='voltage_1_B',
        figsize=(15,5),
        color='blue',
        style='-',
        grid=True,
        )
    plt.xlabel('samples')
    #plt.xlabel('time [ms]')
    plt.ylabel('value')


    # PLOT ALL THE STUFF
    df_firstpulse.plot(
        title="SINGLE PULSE (n={}): CURRENT".format(n),
        y=['current_1_A', 'current_1_B'],
        figsize=(15,5),
        color=['green', 'purple'],
        style=['-', '-'],
        grid=True,
        )
    plt.xlabel('samples')
    #plt.xlabel('time [ms]')
    plt.ylabel('value')

    # PLOT ALL THE STUFF
    df_firstpulse.plot(
        title="SINGLE PULSE (n={}): CURRENT_1_B".format(n),
        y='current_1_B',
        figsize=(15,5),
        color='purple',
        style='-',
        grid=True,
        )
    plt.xlabel('samples')
    #plt.xlabel('time [ms]')
    plt.ylabel('value')








# PRINT MULTIPLE
else:
    # PRINT IT 10 TIMES
    for n in range(n, n+nr_of_prints):
        
        # GET THE Nth PULSE
        df_firstpulse = df_pg.iloc[df_times.iloc[n]['start']:df_times.iloc[n]['end']]

        # PLOT ALL THE STUFF
        fig, ax1 = plt.subplots(figsize=(15,5))
        df_firstpulse.plot(
            title="SINGLE PULSE (n={}): VOLTAGE".format(n),
            y='voltage_1_A',
            ax=ax1,
            color='red',
            style='-',
            grid=True,
            )
        ax1.set_xlabel('samples')
        ax1.set_ylabel('voltage_1_A')
        ax2 = ax1.twinx()
        df_firstpulse.plot(
            y='voltage_1_B',
            ax=ax2,
            color='blue',
            style='-',
            )
        ax2.set_ylabel('voltage_1_B')


plt.show()

In [ ]:
df_pg[df_pg['type'].isnull()]

In [ ]:
df_voltages = df_pg.copy()
df_voltages = df_voltages[df_voltages['type'].isnull()]

#df_voltages['voltage'] = (df_voltages['voltage_1_A'].astype('uint32') << 16) | df_voltages['voltage_1_B'].astype('uint32')     DID NOT WORK

df_voltages['voltage'] = df_voltages.apply(lambda row: int(bin(row['voltage_1_A'])[2:] + bin(row['voltage_1_B'])[2:], 2), axis=1)
df_voltages;


In [ ]:
df_voltages_short = df_voltages[:5000]

# PLOT ALL THE STUFF
df_voltages_short.plot(
    title="PG received data: voltage",
    y='voltage',
    figsize=(15,5),
    color='red',
    style='-',
    grid=True,
    )
plt.xlabel('samples')
#plt.xlabel('time [ms]')
plt.ylabel('value')

# TEST THE COM PORT DETECTION

In [ ]:
from serial.tools import list_ports

def find_serial_ports(vendor_id, psu_product_id, pg_product_id):
    psu_serial = None
    pg_serial = None

    ports = list_ports.comports()
    for port in ports:
        if port.vid == vendor_id:
            if port.pid == psu_product_id:
                psu_serial = port.device
            elif port.pid == pg_product_id:
                pg_serial = port.device
                
    return psu_serial, pg_serial

def find_serial_port(vendor_id, product_id):
    serial = None

    ports = list_ports.comports()
    for port in ports:
        if port.vid == vendor_id and port.pid == product_id:
            serial = port.device

                
    return serial



# Replace vendor_id and product_id with your specific values
vendor_id = 0x6666
pg_product_id = 0x0200
psu_product_id = 0x0100


psu_serial_port, pg_serial_port = find_serial_ports(vendor_id, psu_product_id, pg_product_id)
if psu_serial_port is not None:
    ser_psu = serial.Serial(psu_serial_port)
    print("I FOUND THE PSU! PORT: {}".format(psu_serial_port))
else:
    print("PSU not found.")

if pg_serial_port is not None:
    ser_pg = serial.Serial(pg_serial_port)
    print("I FOUND THE PULSE GENERATOR! PORT: {}".format(pg_serial_port))
else:
    print("PG not found.")

ser_pg.close()
ser_psu.close()

In [ ]:
# FUNCTIONS IN MODULE
def find_serial_port(vendor_id, product_id):
    serial = None

    ports = list_ports.comports()
    for port in ports:
        if port.vid == vendor_id and port.pid == product_id:
            serial = port.device

    return serial

def establish_serial_connection(com_list):
    serials = []

    for com_port in com_list:
        ser = serial.Serial(com_port)
        serials.append(ser)

    return serials

def close_serial_connections(ser_list):
    for ser in ser_list:
        ser.close()

# VARIABLES
vendor_id = 0x6666
pg_product_id = 0x0200
psu_product_id = 0x0100
SERIAL_TEMPSENS_VENDOR_ID = 0x0403  # TEMPERATURE-SENSOR VENDOR ID
SERIAL_TEMPSENS_PRODUCT_ID = 0x6015 # TEMPERATURE-SENSOR PRODUCT ID

# FIND PORTS
psu_serial = find_serial_port(vendor_id, psu_product_id)
pg_serial = find_serial_port(vendor_id, pg_product_id)
temp_serial = find_serial_port(SERIAL_TEMPSENS_VENDOR_ID, SERIAL_TEMPSENS_PRODUCT_ID)

print(psu_serial)
print(pg_serial)
print(temp_serial)

# MAKE LIST
com_list = [psu_serial, pg_serial, temp_serial]

# ESTABLISH ALL CONNECTIONS
my_serials = establish_serial_connection(com_list)
print(my_serials)


# CLOSE ALL CONNECTIONS
close_serial_connections(my_serials)
print(my_serials)

In [ ]:
list_ports.comports()

In [ ]:
ports = list_ports.comports()
for port in ports:
    print("NEXT PORT:")
    print("vendor id\t:", port.vid)
    print("product id:\t", port.pid)
    print("device:\t\t", port.device)
    print("----------------------------\n")